IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M5\NO_FAULT5


In [3]:
guasto=0

VOLO m5 NO FAULT

In [4]:
rcou_m5_nofault = pd.read_csv("RCOU.csv")
rcou_m5_nofault = rcou_m5_nofault.astype("float64")
rcou_m5_nofault = rcou_m5_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m5_nofault = rcou_m5_nofault[((rcou_m5_nofault['C9'] >= potenza) & (rcou_m5_nofault['C10'] >= potenza) & (rcou_m5_nofault['C11'] >= potenza) & (rcou_m5_nofault['C12'] >= potenza) & (rcou_m5_nofault['C13'] >= potenza) & (rcou_m5_nofault['C14']>= potenza))]
rcou_m5_nofault=rcou_m5_nofault.reset_index(drop=True)
display(rcou_m5_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,102505087.0,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0
1,102547919.0,1440.0,1396.0,1417.0,1420.0,1435.0,1401.0
2,102558488.0,1407.0,1441.0,1429.0,1419.0,1409.0,1439.0
3,102568605.0,1458.0,1401.0,1427.0,1434.0,1452.0,1407.0
4,102578804.0,1501.0,1362.0,1438.0,1434.0,1501.0,1362.0
...,...,...,...,...,...,...,...
4949,162697251.0,1552.0,1412.0,1557.0,1405.0,1390.0,1569.0
4950,162729893.0,1554.0,1397.0,1557.0,1393.0,1403.0,1550.0
4951,162753701.0,1563.0,1375.0,1542.0,1403.0,1404.0,1541.0
4952,162764057.0,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0


In [5]:
min=rcou_m5_nofault['TimeUS'][0]
max=rcou_m5_nofault['TimeUS'][len(rcou_m5_nofault)-1]
print(max)
print(min)

162774208.0
102505087.0


In [6]:
xkf1_m5_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m5_nofault = xkf1_m5_nofault.astype("float64")
xkf1_m5_nofault = xkf1_m5_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m5_nofault = xkf1_m5_nofault[((xkf1_m5_nofault['TimeUS'] >= min) & (xkf1_m5_nofault['TimeUS'] <= max	))]
xkf1_m5_nofault = xkf1_m5_nofault.reset_index(drop=True)
print(xkf1_m5_nofault)

           TimeUS  Roll  Pitch    Yaw
0     102513035.0  1.85  -1.49  10.85
1     102523905.0  1.94  -1.53  10.85
2     102535459.0  1.99  -1.53  10.84
3     102545883.0  1.98  -1.51  10.84
4     102555882.0  1.93  -1.48  10.85
...           ...   ...    ...    ...
5479  162727781.0 -2.37  -0.06   6.21
5480  162741639.0 -2.40  -0.08   6.17
5481  162751479.0 -2.43  -0.06   6.12
5482  162761552.0 -2.44  -0.01   6.10
5483  162772111.0 -2.53  -0.03   6.05

[5484 rows x 4 columns]


In [7]:
att_m5_nofault = pd.read_csv("ATT.csv")
att_m5_nofault = att_m5_nofault.astype("float64")
att_m5_nofault = att_m5_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m5_nofault = att_m5_nofault[((att_m5_nofault['TimeUS'] >= min) & (att_m5_nofault['TimeUS'] <= max	))]
att_m5_nofault=att_m5_nofault.reset_index(drop=True)
print(att_m5_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     102515089.0    -0.02  1.85      0.14  -1.49   10.80  10.85
1     102525154.0    -0.04  1.94      0.15  -1.53   10.80  10.85
2     102536919.0    -0.06  1.99      0.15  -1.53   10.80  10.84
3     102547199.0    -0.09  1.98      0.16  -1.51   10.80  10.84
4     102557820.0    -0.10  1.93      0.17  -1.48   10.80  10.85
...           ...      ...   ...       ...    ...     ...    ...
5479  162729207.0     0.38 -2.37     -1.24  -0.06    2.66   6.21
5480  162742824.0     0.42 -2.40     -1.26  -0.08    2.66   6.17
5481  162752885.0     0.47 -2.43     -1.29  -0.06    2.66   6.12
5482  162763219.0     0.52 -2.44     -1.32  -0.01    2.66   6.10
5483  162773559.0     0.58 -2.53     -1.35  -0.03    2.66   6.05

[5484 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m5_nofault)>len(xkf1_m5_nofault):
    to_add=att_m5_nofault[len(xkf1_m5_nofault):]
    att_m5_nofault=att_m5_nofault[:len(xkf1_m5_nofault)]

if len(xkf1_m5_nofault)>len(att_m5_nofault):
    to_add=xkf1_m5_nofault[len(att_m5_nofault):]

for idx,i in enumerate(att_m5_nofault['Roll']):
    if(xkf1_m5_nofault['Roll'][idx] != i):
        att_m5_nofault['Roll'][idx] = ((att_m5_nofault['Roll'][idx] + xkf1_m5_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m5_nofault['Pitch']):
    if(xkf1_m5_nofault['Pitch'][idx] != i):
        att_m5_nofault['Pitch'][idx] = ((att_m5_nofault['Pitch'][idx] + xkf1_m5_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m5_nofault['Yaw']):
    if(xkf1_m5_nofault['Yaw'][idx] != i):
        att_m5_nofault['Yaw'][idx] = ((att_m5_nofault['Yaw'][idx] + xkf1_m5_nofault['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m5_nofault=pd.concat([att_m5_nofault,to_add])

print(att_m5_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     102515089.0    -0.02  1.85      0.14  -1.49   10.80  10.85
1     102525154.0    -0.04  1.94      0.15  -1.53   10.80  10.85
2     102536919.0    -0.06  1.99      0.15  -1.53   10.80  10.84
3     102547199.0    -0.09  1.98      0.16  -1.51   10.80  10.84
4     102557820.0    -0.10  1.93      0.17  -1.48   10.80  10.85
...           ...      ...   ...       ...    ...     ...    ...
5479  162729207.0     0.38 -2.37     -1.24  -0.06    2.66   6.21
5480  162742824.0     0.42 -2.40     -1.26  -0.08    2.66   6.17
5481  162752885.0     0.47 -2.43     -1.29  -0.06    2.66   6.12
5482  162763219.0     0.52 -2.44     -1.32  -0.01    2.66   6.10
5483  162773559.0     0.58 -2.53     -1.35  -0.03    2.66   6.05

[5484 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m5_nofault = pd.read_csv("ESC_0.csv")
esc_0_m5_nofault = esc_0_m5_nofault.astype("float64")
esc_0_m5_nofault = esc_0_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m5_nofault = esc_0_m5_nofault[((esc_0_m5_nofault['TimeUS'] >= min) & (esc_0_m5_nofault['TimeUS'] <= max))]
esc_0_m5_nofault=esc_0_m5_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m5_nofault = pd.read_csv("ESC_1.csv")
esc_1_m5_nofault = esc_1_m5_nofault.astype("float64")
esc_1_m5_nofault = esc_1_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m5_nofault = esc_1_m5_nofault[((esc_1_m5_nofault['TimeUS'] >= min) & (esc_1_m5_nofault['TimeUS'] <= max))]
esc_1_m5_nofault=esc_1_m5_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m5_nofault = pd.read_csv("ESC_2.csv")
esc_2_m5_nofault = esc_2_m5_nofault.astype("float64")
esc_2_m5_nofault = esc_2_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m5_nofault = esc_2_m5_nofault[((esc_2_m5_nofault['TimeUS'] >= min) & (esc_2_m5_nofault['TimeUS'] <= max))]
esc_2_m5_nofault=esc_2_m5_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m5_nofault = pd.read_csv("ESC_3.csv")
esc_3_m5_nofault = esc_3_m5_nofault.astype("float64")
esc_3_m5_nofault = esc_3_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m5_nofault = esc_3_m5_nofault[((esc_3_m5_nofault['TimeUS'] >= min) & (esc_3_m5_nofault['TimeUS'] <= max))]
esc_3_m5_nofault=esc_3_m5_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m5_nofault = pd.read_csv("ESC_4.csv")
esc_4_m5_nofault = esc_4_m5_nofault.astype("float64")
esc_4_m5_nofault = esc_4_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m5_nofault = esc_4_m5_nofault[((esc_4_m5_nofault['TimeUS'] >= min) & (esc_4_m5_nofault['TimeUS'] <= max))]
esc_4_m5_nofault=esc_4_m5_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m5_nofault = pd.read_csv("ESC_5.csv")
esc_5_m5_nofault = esc_5_m5_nofault.astype("float64")
esc_5_m5_nofault = esc_5_m5_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m5_nofault = esc_5_m5_nofault[((esc_5_m5_nofault['TimeUS'] >= min) & (esc_5_m5_nofault['TimeUS'] <= max))]
esc_5_m5_nofault=esc_5_m5_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m5_nofault = pd.read_csv("ESC_0.csv")\nesc_0_m5_nofault = esc_0_m5_nofault.astype("float64")\nesc_0_m5_nofault = esc_0_m5_nofault.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m5_nofault = esc_0_m5_nofault[((esc_0_m5_nofault[\'TimeUS\'] >= min) & (esc_0_m5_nofault[\'TimeUS\'] <= max))]\nesc_0_m5_nofault=esc_0_m5_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m5_nofault = pd.read_csv("ESC_1.csv")\nesc_1_m5_nofault = esc_1_m5_nofault.astype("float64")\nesc_1_m5_nofault = esc_1_m5_nofault.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m5_nofault = esc_1_m5_nofault[((esc_1_m5_nofault[\'TimeUS\'] >= min) & (esc_1_m5_nofault[\'TimeUS\'] <= max))]\nesc_1_m5_nofault=esc_1_m5_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\n

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m5_nofault = pd.read_csv("IMU_0.csv")
imu_0_m5_nofault = imu_0_m5_nofault.astype("float64")
imu_0_m5_nofault = imu_0_m5_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m5_nofault = imu_0_m5_nofault[((imu_0_m5_nofault['TimeUS'] >= min) & (imu_0_m5_nofault['TimeUS'] <= max))]

imu_1_m5_nofault = pd.read_csv("IMU_1.csv")
imu_1_m5_nofault = imu_1_m5_nofault.astype("float64")
imu_1_m5_nofault = imu_1_m5_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m5_nofault = imu_1_m5_nofault[((imu_1_m5_nofault['TimeUS'] >= min) & (imu_1_m5_nofault['TimeUS'] <= max))]

imu_2_m5_nofault = pd.read_csv("IMU_2.csv")
imu_2_m5_nofault = imu_2_m5_nofault.astype("float64")
imu_2_m5_nofault = imu_2_m5_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m5_nofault = imu_2_m5_nofault[((imu_2_m5_nofault['TimeUS'] >= min) & (imu_2_m5_nofault['TimeUS'] <= max))]

imu_m5_nofault = pd.concat((imu_0_m5_nofault, imu_1_m5_nofault, imu_2_m5_nofault))
imu_m5_nofault=imu_m5_nofault.groupby(imu_m5_nofault.TimeUS, as_index=False).mean()

display(imu_m5_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,102506599.0,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544
1,102509578.0,-0.199651,-0.022153,0.011422,-0.104806,0.072470,-9.512378
2,102511769.0,0.084841,-0.071096,-0.003225,-0.121335,0.205762,-9.454771
3,102514770.0,0.295913,-0.067083,-0.007073,-0.237170,0.049783,-9.474679
4,102517655.0,-0.020380,0.062274,0.011793,-0.314133,0.005627,-9.648276
...,...,...,...,...,...,...,...
20824,162763113.0,-0.033571,0.084760,-0.042827,0.069566,0.555606,-9.501016
20825,162765819.0,0.120113,0.012521,-0.068512,0.072377,0.397154,-9.557518
20826,162768625.0,0.021385,-0.016392,-0.085739,0.055393,0.223394,-9.623663
20827,162770890.0,-0.150573,-0.009726,-0.084213,0.030201,0.249535,-9.643305


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m5_nofault = pd.merge_ordered(imu_m5_nofault,att_m5_nofault)
m5_nofault=m5_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m5_nofault = pd.merge_ordered(m5_nofault, esc_0_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_1_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_2_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_3_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_4_m5_nofault)
m5_nofault = pd.merge_ordered(m5_nofault, esc_5_m5_nofault)
m5_nofault=m5_nofault.fillna(method="ffill").fillna(method="bfill")'''
m5_nofault = pd.merge_ordered(m5_nofault, rcou_m5_nofault)
m5_nofault=m5_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m5_nofault["TimeUS"] = m5_nofault["TimeUS"] - m5_nofault.iloc[0]["TimeUS"]
m5_nofault["TimeUS"] = m5_nofault["TimeUS"].astype(int)
m5_nofault["Guasto"] = guasto

In [13]:
last_cell = m5_nofault.iloc[-1, m5_nofault.columns.get_loc('TimeUS')]
print(last_cell)

60269121


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
21091  60256987
21092  60259844
21093  60262701
21094  60265558
21095  60268415

[21096 rows x 1 columns]


In [16]:
m5_nofault_final = pd.merge_ordered(m5_nofault,df)
m5_nofault_final=m5_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m5_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY      AccZ  \
0             0 -0.364870  0.051198  0.015439 -0.196391 -0.273725 -9.656544   
1          1512 -0.364870  0.051198  0.015439 -0.196391 -0.273725 -9.656544   
2          2857 -0.364870  0.051198  0.015439 -0.196391 -0.273725 -9.656544   
3          4491 -0.199651 -0.022153  0.011422 -0.104806  0.072470 -9.512378   
4          5714 -0.199651 -0.022153  0.011422 -0.104806  0.072470 -9.512378   
...         ...       ...       ...       ...       ...       ...       ...   
52347  60265803 -0.150573 -0.009726 -0.084213  0.030201  0.249535 -9.643305   
52348  60268204 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   
52349  60268415 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   
52350  60268472 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   
52351  60269121 -0.152369  0.006015 -0.076671  0.028353  0.414205 -9.604354   

       DesRoll  Roll  DesPitch  Pitch  DesYaw    Ya

In [17]:
m5_nofault_final = m5_nofault_final[m5_nofault_final.TimeUS.isin(to_be)]

In [18]:
m5_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
2,2857,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
4,5714,-0.199651,-0.022153,0.011422,-0.104806,0.072470,-9.512378,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
6,8571,0.084841,-0.071096,-0.003225,-0.121335,0.205762,-9.454771,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
9,11428,0.295913,-0.067083,-0.007073,-0.237170,0.049783,-9.474679,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52338,60256987,-0.160556,0.065735,-0.040423,-0.010292,0.480986,-9.432996,0.47,-2.43,-1.29,-0.06,2.66,6.12,1563.0,1375.0,1542.0,1403.0,1404.0,1541.0,0.0
52342,60259844,-0.033571,0.084760,-0.042827,0.069566,0.555606,-9.501016,0.52,-2.44,-1.32,-0.01,2.66,6.10,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
52344,60262701,0.120113,0.012521,-0.068512,0.072377,0.397154,-9.557518,0.52,-2.44,-1.32,-0.01,2.66,6.10,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
52346,60265558,0.021385,-0.016392,-0.085739,0.055393,0.223394,-9.623663,0.52,-2.44,-1.32,-0.01,2.66,6.10,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0


In [19]:
from datetime import timedelta
m5_nofault_final=m5_nofault_final.reset_index(drop=True)
m5_nofault_final['TimeUS'] = pd.to_datetime(m5_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m5_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
1,00:00:00.002857,-0.364870,0.051198,0.015439,-0.196391,-0.273725,-9.656544,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
2,00:00:00.005714,-0.199651,-0.022153,0.011422,-0.104806,0.072470,-9.512378,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
3,00:00:00.008571,0.084841,-0.071096,-0.003225,-0.121335,0.205762,-9.454771,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
4,00:00:00.011428,0.295913,-0.067083,-0.007073,-0.237170,0.049783,-9.474679,-0.02,1.85,0.14,-1.49,10.80,10.85,1385.0,1404.0,1405.0,1384.0,1392.0,1397.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21091,00:01:00.256987,-0.160556,0.065735,-0.040423,-0.010292,0.480986,-9.432996,0.47,-2.43,-1.29,-0.06,2.66,6.12,1563.0,1375.0,1542.0,1403.0,1404.0,1541.0,0.0
21092,00:01:00.259844,-0.033571,0.084760,-0.042827,0.069566,0.555606,-9.501016,0.52,-2.44,-1.32,-0.01,2.66,6.10,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
21093,00:01:00.262701,0.120113,0.012521,-0.068512,0.072377,0.397154,-9.557518,0.52,-2.44,-1.32,-0.01,2.66,6.10,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0
21094,00:01:00.265558,0.021385,-0.016392,-0.085739,0.055393,0.223394,-9.623663,0.52,-2.44,-1.32,-0.01,2.66,6.10,1547.0,1391.0,1519.0,1426.0,1357.0,1572.0,0.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m5_nofault_final)/350)):
        V1=m5_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_5748\1153984652.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_5748\1153984652.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.062372,0.011086,0.728895,0.122247,0.0,91.0,11.309668,3.303893,0.026686,0.007082,-0.177517,0.088170,96.0,70.0,4.177586,2.744264,0.087336,0.003749,-1.046062,0.106611,65.0,55.0,4.164788,0.854312,-0.241336,0.059795,0.116931,0.343318,108.0,113.0,20.633215,11.512607,0.254905,0.102266,-0.487814,0.408596,118.0,119.0,8.595355,7.022741,-10.214780,0.571122,-0.086027,10.242618,0.0,114.0,105.789495,30.074047,0.827664,2.076401,-1.356350,1.659974,108.0,107.0,7.584688,6.835745,-0.367607,0.496851,-1.321972,0.794084,100.0,98.0,31.513793,1.108642,12.851681,3.240768,-1.556174,12.976796,104.0,76.0,0.571437,-0.679453,1544.512821,2357.399121,3.583290,1545.273612,105.0,104.0,869.526370,690.534517,1438.250712,1807.051249,-0.840452,1438.876998,117.0,119.0,1443.981225,1362.384182,1500.219373,1051.086024,1.771754,1500.568645,114.0,110.0,1088.333720,606.531277,1488.213675,1902.574212,0.239426,1488.850932,108.0,109.0,514.327001,501.026149,1490.638177,2060.191567,-0.814630,1491.327092,111.0,110.0,1603.047386,1163.553607,1496.250712,2798.942678,0.036596,1497.183076,107.0,109.0,1561.285225,1415.413469,0
1,1.0,-0.062231,0.011061,0.732446,0.122075,0.0,102.0,11.451014,3.247529,0.026223,0.007137,-0.176852,0.088343,93.0,67.0,4.051908,1.984847,0.087392,0.003739,-1.041186,0.106612,63.0,59.0,4.204048,0.660200,-0.241214,0.059630,0.126055,0.342994,109.0,114.0,23.131326,11.520866,0.255303,0.102030,-0.480917,0.408558,103.0,99.0,8.264990,7.784373,-10.212998,0.570613,-0.078654,10.240816,0.0,114.0,105.217752,29.468904,0.820426,2.088924,-1.362548,1.660147,108.0,107.0,7.917468,6.983656,-0.367017,0.495558,-1.315793,0.793002,103.0,101.0,31.463264,0.939003,12.860284,3.257589,-1.556702,12.985963,109.0,107.0,0.652385,-1.390682,1544.639205,2356.305353,3.592372,1545.399587,101.0,100.0,928.674622,697.922927,1438.093750,1810.575231,-0.842686,1438.721330,111.0,113.0,1359.534669,1317.765553,1500.332386,1052.587210,1.771682,1500.682134,118.0,117.0,1072.068445,620.495861,1488.096591,1901.979248,0.233394,1488.733703,115.0,117.0,567.937441,496.612078,1490.681818,2054.992489,-0.808646,1491.368981,103.0,102.0,1501.323867,1139.099130,1496.215909,2791.394846,0.041526,1497.145788,101.0,103.0,1529.555531,1381.445247,0
2,2.0,-0.062510,0.011057,0.731551,0.122202,0.0,111.0,11.443601,3.035146,0.026083,0.007124,-0.170130,0.088226,116.0,105.0,4.039487,1.793598,0.087451,0.003730,-1.036406,0.106616,71.0,59.0,4.243139,0.699154,-0.241395,0.059472,0.132588,0.342892,108.0,113.0,25.047807,11.418497,0.255737,0.101806,-0.474286,0.408558,105.0,109.0,11.763111,7.988073,-10.211328,0.569976,-0.070785,10.239120,0.0,110.0,104.607530,28.782014,0.813229,2.101272,-1.368786,1.660320,104.0,103.0,8.251755,7.120035,-0.366431,0.494271,-1.309608,0.791923,96.0,95.0,31.405991,0.876648,12.868839,

In [23]:
path_file = path_file.replace(r"M5\NO_FAULT5", "")
os.chdir(path_file)
df_merged.to_csv('m5_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
